In [3]:
import cvxpy as cp
import numpy as np

# given parameters
m = 0.22;
M = 1.3282;
f0 = 22.915;
f1 = 0.007056;
l = 0.304;
J = 0.004963;
g = 9.8;

# determined domain
a = np.pi/3  # x1 \in [-a a]
b = 1        # x2 \in [-b b]
Premise_var_x1 = np.array([[-a, 0, a]])
Premise_var_x2 = np.array([[-b, b]])

n_r = np.size(Premise_var_x1)*np.size(Premise_var_x2)  # number of rules
n_s = 4  # size of A[i] matrix
n_u = 1  # size of B[i] matrix

# Define A and B matrices
A = []
B = []

A = []
for i in range(np.size(Premise_var_x1)):
  for j in range(np.size(Premise_var_x2)):
    x1 = Premise_var_x1[0,i]
    x2 = Premise_var_x2[0,j]
    if x1 == 0:
      ad = (M+m)*(J+m*l**2) - (m**2*l**2);
      a21d = (M+m)*m*g*(l/a);
      a22d = -f1*(M+m)/a;
      a24d = f0*m*(l/a);
      a41d = -m**2*g*(l**2/a);
      a42d = f1*m*(l/a);
      a44d = -f0*(J+m*l**2)/a;
      b2d = -m*(l/a);
      b4d = (J+m*l**2)/a;
    else:
      ad = (M+m)*(J+m*l**2)-m**2*l**2*np.cos(x1)**2
      a21d = (np.sin(x1)/x1)*(M+m)*m*g*(l/ad)
      a22d = (-f1*(M+m)-(m**2*l**2*x2*np.sin(x1)*np.cos(x1)))/ad
      a24d = f0*m*l*np.cos(x1)/ad
      a41d = -(np.sin(x1)*np.cos(x1)/x1)*((m**2*g*l**2)/ad)
      a42d = (f1*m*l*np.cos(x1)+((J+m*l**2)*(m*l*x2)*np.sin(x1)))/ad
      a44d = -f0*((J+m*l**2)/ad)
      b2d = -m*l*np.cos(x1)/ad
      b4d = (J+m*l**2)/ad

    A.append(np.array([[0, 1, 0, 0],
                  [a21d, a22d, 0, a24d],
                  [0, 0, 0, 1],
                  [a41d, a42d, 0, a44d]]))

    B.append(np.array([[0],
                    [b2d],
                    [0],
                    [b4d]]))


# decision variables
Q = cp.Variable((n_s, n_s), symmetric=True)
Y = [cp.Variable((n_u, n_s)) for _ in range(n_r)]
I = np.identity(n_s)
#gamma = cp.Variable((1, 1))
# LMI constraints
constraints = []

# LMI condition for stabilization of each sub system
for i in range(n_r):
    Ai = A[i]
    Bi = B[i]
    constraints.append((Ai @ Q + Q @ Ai.T + Bi @ Y[i] + Y[i].T @ Bi.T) << 0)

# LMI condition for stabilization of joint systems (refer [1])
# case 1
for j in range(1, n_r):
    Ai = A[0]
    Aj = A[j]
    Bi = B[0]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[0] + Y[0].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# case 2
for j in range(2, n_r):
    Ai = A[1]
    Aj = A[j]
    Bi = B[1]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[1] + Y[1].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# case 3
for j in range(3, n_r):
    Ai = A[2]
    Aj = A[j]
    Bi = B[2]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[2] + Y[2].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# Positive semidefinite constraint on Q
constraints.append(Q - 0.1*I>> 0) # 0.1 is for avoiding numerical problems

problem = cp.Problem(cp.Minimize(0), constraints)
print(problem.is_qp())
problem.solve(verbose = True)  # you can choose SDP solvers (recommend "SCS" solver)


ch = [c.dual_value for c in constraints]

K = [Y[i].value @ np.linalg.inv(Q.value) for i in range(n_r)]

for i in range(n_r):
  print(f'K{{{i+1}}}=', K[i])

True
                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Aug 14 11:55:19 AM: Your problem has 40 variables, 304 constraints, and 0 parameters.
(CVXPY) Aug 14 11:55:19 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 14 11:55:19 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 14 11:55:19 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 14 11:55:19 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 14 11:55:19 AM: Compiling problem (target solver=SCS)

In [21]:

for i in range(6):
  [a, b] = np.linalg.eig(A[0]+B[0]*K[i])
  print(a)
  [a, b] = np.linalg.eig(A[1]+B[1]*K[i])
  print(a)
  [a, b] = np.linalg.eig(A[2]+B[2]*K[i])
  print(a)
  [a, b] = np.linalg.eig(A[3]+B[3]*K[i])
  print(a)
  [a, b] = np.linalg.eig(A[4]+B[4]*K[i])
  print(a)
  [a, b] = np.linalg.eig(A[5]+B[5]*K[i])
  print(a)

# print( np.linalg.eig(A[0]+B[0]*K[1]))
# print( np.linalg.eig(A[0]+B[0]*K[2]))
# print( np.linalg.eig(A[0]+B[0]*K[3]))
# print( np.linalg.eig(A[0]+B[0]*K[4]))
# print( np.linalg.eig(A[0]+B[0]*K[5]))



[-3.05984279+7.2411132j -3.05984279-7.2411132j -7.91978152+0.j
 -0.05081765+0.j       ]
[-3.0507487 +7.2903235j -3.0507487 -7.2903235j -7.83614539+0.j
 -0.05081768+0.j       ]
[-0.88530109+2.87620128j -0.88530109-2.87620128j -0.0393817 +0.04944843j
 -0.0393817 -0.04944843j]
[-0.88530109+2.87620128j -0.88530109-2.87620128j -0.0393817 +0.04944843j
 -0.0393817 -0.04944843j]
[-3.0507487 +7.2903235j -3.0507487 -7.2903235j -7.83614539+0.j
 -0.05081768+0.j       ]
[-3.05984279+7.2411132j -3.05984279-7.2411132j -7.91978152+0.j
 -0.05081765+0.j       ]
[-3.05911161+7.20678198j -3.05911161-7.20678198j -8.14626557+0.j
 -0.05065572+0.j        ]
[-3.0514536 +7.25550683j -3.0514536 -7.25550683j -8.05975727+0.j
 -0.05065575+0.j        ]
[-0.90142348+2.88331855j -0.90142348-2.88331855j -0.03986272+0.04943624j
 -0.03986272-0.04943624j]
[-0.90142348+2.88331855j -0.90142348-2.88331855j -0.03986272+0.04943624j
 -0.03986272-0.04943624j]
[-3.0514536 +7.25550683j -3.0514536 -7.25550683j -8.05975727+0.j
 -0.0